# Docker QuickGuide

![](statics/docker_concept.png)

**What is docker?**

Docker is an operating system for containers. Similar to how a virtual machine virtualizes (removes the need to directly manage) server hardware, containers virtualize the operating system of a server. Docker is installed on each server and provides simple commands you can use to build, start, or stop containers.

Quick tutorial: https://www.youtube.com/watch?v=gAkwW2tuIqE

## Docker x MLflow | From start to finish

The following demonstration is done entirely on CLI

### **1. Download Docker Desktop**

https://www.docker.com/products/docker-desktop/

### **2. Connect to the desire mlflow tracking server**

Mac/Linux/Unix:

`export MLFLOW_TRACKING_URI=sqlite:///mlruns.db` (replace sqlite:///mlruns.db with your tracking server)

Windows:

`set MLFLOW_TRACKING_URI=sqlite:///mlruns.db` <br>
or use `$env:MLFLOW_TRACKING_URI="sqlite:///mlruns.db` if the above doesn't work

### **3. Create docker image**

`mlflow models build-docker --model-uri "models:/iris_model/1" --name "iris"` <br>

>**models:/iris_model/1 >> "models:/\<registered model name>/\<model version>"** <br>
>**--name "iris" >> --name "name of the image"**


<br>

![image.png](statics/docker_desktop_image.png)

**After the image finish building, you will be able to see it in the Docker Desktop**

#### **4. Run the container**

`docker run -p 5002:8080 iris`

**Noted that it is necessary to use -p flag which serves as port forwarding for it to be able to run locally**

>**Port 5002 represents the local port, this is the port that you will be running the model on your local device**

>**Port 8080 represents the container port**

>**"iris" is the name we defined in section 3**

![](statics/runningcontainer.png)

You will be able to see the container running in your docker desktop (PORT 5002 is the local port that you will run your container on)

#### **5. Use the model**

##### **5.1 Curl**
Now that the container is running, you can access the model is various ways. The first being using the `curl` command in CLI

`curl http://127.0.0.1:5002/invocations -H 'Content-Type: application/json' -d '{"dataframe_split": {"data": [[6.1, 2.8, 4.7, 1.2], [5.7, 3.8, 1.7, 0.3]]}}'`

> http://127.0.0.1:5002/invocations >> 5002 is the local port that we are running the container on, /invocations is the endpoint for mlflow conventions

**\*Note\* Since windows does not support single quote, you have to use this command instead if you are using windows**

`curl http://127.0.0.1:5002/invocations -H "Content-Type: application/json" -d "{\"dataframe_split\": {\"data\": [[6.1, 2.8, 4.7, 1.2], [5.7, 3.8, 1.7, 0.3]]}}"`

The curl command accepts several content-type and keys, for more information: https://mlflow.org/docs/latest/models.html#local-model-deployment

##### **5.2 request**
Alternatively, you can use the **request** library to make a post request to the corresponding endpoint

```python
import requests

endpoint = "http://localhost:5002/invocations" # make sure the port number matches the one you input in terminal
data = {
    "dataframe_split": {  
        "data": [[6.1, 2.8, 4.7, 1.2], [5.7, 3.8, 1.7, 0.3]]   
    }
}
# do a post request
response = requests.post(endpoint, json=data)
print(response.json())
```

#### **6. Pushing the docker image**